In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

# 함수 정의

In [ ]:
"""
데이터 셋 생성 함수 >>>>>>>>>>>>>>> 여기서 windowsize= 3은 3시간 치의 
데이터로 다음 데이터를 예측하는 것(하이퍼파라미터) 변경 가능
"""

# 성능확인
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

def confirm_result(test_label,test_predict):
    MAE = mean_absolute_error(test_label,test_predict)
    RMSE = np.sqrt(mean_squared_error(test_label,test_predict))
    MSE = RMSE**2
    R2 = r2_score(test_label,test_predict)

    Result = pd.DataFrame(data=[MAE, RMSE,MSE, R2],
                        index= ['MAE', 'RMSE','MSE','R2'],
                        columns =['Results'])
  
    return Result

# 데이터셋(Train) 생성함수 정의
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size - 72):
        feature_list.append(np.array(data.iloc[i:i + window_size]))
        label_list.append(np.array(label.iloc[i + 72 + window_size]))
    return np.array(feature_list), np.array(label_list)

# 데이터셋(Test) 생성함수 정의
def make_dataset_test(data, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i + window_size]))
    return np.array(feature_list)

# 정규화 된 값 돌리기 위한 함수
# 정규화된 값을 원래의 값으로 되돌린다
# 정규화하기 이전의 org_x값과 되돌리고 싶은 x를 입력하면 역정규화된 값을 리턴한다
def reverse_min_max_scaling(org_x, x):
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min() + 1e-7)) + org_x_np.min()

## 전처리 된 DATASET 불러오기

In [ ]:
df_all = pd.read_csv('DF_ALL_20_21(지면온도).csv', encoding='cp949') # 24개월 데이터 모두

In [ ]:
df_all['date'] = pd.to_datetime(df_all['date'])
df_all_date = df_all['date']

df_all = df_all.set_index('date')
df = df_all

## 정규화

In [ ]:
#####정규화#####
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

scale_cols = df.columns

df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled =pd.DataFrame(df_scaled)
df_scaled = df_scaled.set_index(df_all_date)

In [ ]:
# Test 사이즈 지정
train = df_scaled[:-168] #ex) 240은 240시간 = 10일
test = df_scaled[-96:]

## TRAIN TEST SPLIT

In [ ]:
WINDOWSIZE = 24

#### (windowsize)시간의 데이터로 1시간을 예측
from sklearn.model_selection import train_test_split

feature_cols = range(1,20)
label_cols = [0] ##

x_train = train[feature_cols]
y_train = train[label_cols]

x_train, y_train = make_dataset(x_train, y_train, WINDOWSIZE) #windowsize 변경하기 
print(x_train.shape)

x_test = test[feature_cols]
x_test = make_dataset_test(x_test, WINDOWSIZE)

# SVR

In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf',C=50000,gamma=0.01)
pipeline_svr = Pipeline([
    ('standard_scaler', StandardScaler()), 
    ('pca', PCA()), 
    ('model', svr)
])

pipeline.fit(x_train,y_train)

In [ ]:
svr_pred = pipeline_svr.fit(x_train, y_train).predict(x_test)
confirm_result(y_test, svr_pred)

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(y_test, label = 'actual')
plt.plot(svr_pred, label = 'prediction')
plt.legend()
plt.show()

# KNR

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knr = KNeighborsRegressor()
knr.n_neighbors = 5
pipeline_knr = Pipeline([
    ('standard_scaler', StandardScaler()), 
    ('pca', PCA()), 
    ('model', knr)
])
pipeline_knr.fit(X_train,y_train)

In [ ]:
knr_pred = pipeline_knr.fit(x_train, y_train).predict(x_test)
confirm_result(y_test, knr_pred)

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(y_test, label = 'actual')
plt.plot(knr_pred, label = 'prediction')
plt.legend()
plt.show()

# XGBoost

In [ ]:
import xgboost as xgb
xg_reg=xgb.XGBRegressor(objective='reg:squarederror',booster='gbtree',colsample_bytree=0.85,
                        learning_rate=0.2,max_depth=5,alpha=1,n_estimators=50000)
pipeline_xgr = Pipeline([
    ('standard_scaler', StandardScaler()), 
    ('pca', PCA()), 
    ('model', xg_reg)
])

In [ ]:
xgr_pred = pipeline_xgr.fit(X_train, y_train).predict(X_test)
confirm_result(y_test, xgr_pred)

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(y_test, label = 'actual')
plt.plot(knr_pred, label = 'prediction')
plt.legend()
plt.show()

# LGBM

In [ ]:
from lightgbm import LGBMRegressor
lgbm_reg = LGBMRegressor(booster='gbtree',colsample_bytree=0.75,learning_rate=0.1,max_depth=5,
                       alpha=1,n_estimators=10000)
pipeline_lgbm = Pipeline([
    ('standard_scaler', StandardScaler()), 
    ('pca', PCA()), 
    ('model', lgbm_reg)
])

In [ ]:
lgbm_pred = pipeline_lgbm.fit(X_train, y_train).predict(X_test)
confirm_result(y_test, lgbm_pred)

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(y_test, label = 'actual')
plt.plot(lgbm_pred, label = 'prediction')
plt.legend()
plt.show()

# LSTM

In [ ]:
"""
#####모델 구성#####
"""
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM

model_LSTM = Sequential()
model_LSTM.add(LSTM(32, input_shape=(x_train.shape[1], x_train.shape[2]), 
                            activation='tanh', return_sequences=True)) # 층을 여러개 쓰려면 True로 변경
model_LSTM.add(LSTM(64, input_shape=(x_train.shape[1], x_train.shape[2]), 
                            activation='tanh', return_sequences=True)) # 층을 여러개 쓰려면 True로 변경
model_LSTM.add(LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2]), 
                            activation='tanh', return_sequences=False)) # 층을 여러개 쓰려면 True로 변경
model_LSTM.add(Dense(64))
model_LSTM.add(Dense(32))
model_LSTM.add(Dense(1))
model_LSTM.summary()

In [ ]:
"""
##### 모델 실행 #####
"""
import os

model_LSTM.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)

print('train:', x_train.shape, y_train.shape, '  valid:',x_valid.shape, y_valid.shape)

history = model_LSTM.fit(x_train, y_train, epochs=10, batch_size=128, 
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop])

pred_LSTM = model_LSTM.predict(x_test)


loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

print(confirm_result(y_test, pred_LSTM))

In [ ]:
val_loss_acc = model.evaluate(x_train, y_train)
test_loss_acc = model.evaluate(x_valid, y_valid)
print('  val_loss_acc:', round(val_loss_acc, 4), '  test_loss_acc:', round(test_loss_acc, 4),'  loss:',  round(loss[-1], 4), '  val_loss:',round(val_loss[-1], 4))

# GRU

In [ ]:
"""
#####모델 구성#####
"""
from tensorflow.keras.layers import GRU

model_GRU = Sequential()
model_GRU.add(GRU(units=32, activation="relu", return_sequences=True, input_shape =(x_train.shape[1], x_train.shape[2])))
model_GRU.add(GRU(units=64, activation="relu", return_sequences=True))
model_GRU.add(GRU(units=128, activation="relu", return_sequences=False))
model_GRU.add(Dense(units = 64))                
model_GRU.add(Dense(units = 32))
model_GRU.add(Dense(units = 1))
model_GRU.summary()

In [ ]:
"""
##### 모델 실행 #####
"""
import os
#컴파일링
model_GRU.compile(optimizer='adam',loss='mean_squared_error')
 #피팅

# early_stop = EarlyStopping(monitor='val_loss', patience=10)
history = model_GRU.fit(x_train, train_label,epochs=50,batch_size=16)

pred_GRU = model_GRU.predict(y_train)

loss = history.history['loss']
epochs = range(len(loss))

print(confirm_result(y_test, pred_GRU))

In [ ]:
val_loss_acc = model.evaluate(x_train, y_train)
test_loss_acc = model.evaluate(x_valid, y_valid)
print('  val_loss_acc:', round(val_loss_acc, 4), '  test_loss_acc:', round(test_loss_acc, 4),'  loss:',  round(loss[-1], 4), '  val_loss:',round(val_loss[-1], 4))

# CNN-LSTM

In [ ]:
""" 
#####시퀀스 데이터셋 구성##### 
"""
import tensorflow as tf

def windowed_dataset(series, window_size, batch_size, shuffle):
  series = tf.expand_dims(series, axis=-1)
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size + 48, shift = 1, drop_remainder = True)
  ds = ds.flat_map(lambda w: w.batch(window_size + 48))      # window_size 시간의 데이터로 다음 1시간을 예측
  if shuffle:
    ds = ds.shuffle(1000)
  ds = ds.map(lambda w: (w[:-48], w[-24]))

  return ds.batch(batch_size).prefetch(1)

In [ ]:
"""
#####모델 구성#####
"""
from tensorflow.keras.layers import Conv1D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model_CNN_LSTM = Sequential([
    Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu', input_shape=[WINDOW_SIZE, 1]),
    LSTM(16, activation='tanh'), Dense(16, activation='relu'), Dense(1),
])

"""
#####모델 생성#####
"""
model_CNN_LSTM.compile(loss='mse', optimizer='Adam', metrics=['mse'],run_eagerly=True)

In [ ]:
"""
#####모델 학습#####
"""
checkpoint = ModelCheckpoint(path, save_weights_only=True, save_best_only=True, monitor='val_loss', verbose=1, )  # 최적 loss 값을 갖는 모델 저장
history = model_CNN_LSTM.fit(train_data, validation_data=(test), epochs=15, batch_size=32, callbacks=[checkpoint])

pred_CNN_LSTM = model.predict(test)
confirm_result(test, pred_CNN_LSTM)

## 손실함수 그래프 확인

In [ ]:
#####훈련데이터 검증데이터 손실함수 그래프 나타내기
## 각 모델별로 따로 그리기

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

plt.figure(figsize=(20,10))
plt.plot(pred, label = 'actual')
plt.plot(test_label, label = 'prediction')
plt.legend()
plt.show()

print(confirm_result(pred,test_label))